**Importation des données**

In [19]:
from google.colab import files

# Cette commande va ouvrir un bouton pour uploader un fichier
uploaded = files.upload()

Saving colon_cancer.csv to colon_cancer (1).csv


**Importation**

In [20]:
# --- Imports ---
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
import joblib

print("Imports terminés")


Imports terminés


**Chargement et prétraitement des données**

In [21]:
# Chargement et prétraitement des données

# Chargement
data = pd.read_csv("colon_cancer.csv", sep=";")
data["tissue_status"] = data["tissue_status"].map({"normal": 0, "tumoral": 1})

# Gestion des valeurs manquantes
if data.isnull().sum().sum() > 0:
    data = data.dropna()

# Séparation features / cible
X = data.drop(columns=["id_sample", "tissue_status"])
y = data["tissue_status"]

print(f"Données chargées et prétraitées ({data.shape[0]} échantillons)")


Données chargées et prétraitées (804 échantillons)


**Sélection du meilleur gène par p-value**

In [22]:
## Sélection du meilleur gène basé sur la p-value

p_values = {}

for gene in X.columns:
    normal = X[gene][y == 0]
    tumoral = X[gene][y == 1]
    t_stat, p_val = ttest_ind(normal, tumoral, equal_var=False)
    p_values[gene] = p_val

# Gène avec la p-value minimale
best_gene = min(p_values, key=p_values.get)

print(f"Meilleur gène sélectionné : {best_gene} (p-value : {p_values[best_gene]:.6f})")


Meilleur gène sélectionné : UGP2 (p-value : 0.000000)


**Préparation des données pour la modélisation**

In [23]:
## Préparation des données pour la modélisation

# Sélection du meilleur gène
X_best = X[[best_gene]]

# Normalisation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_best)

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print(f"Données prêtes pour l'entraînement (train: {X_train.shape[0]}, test: {X_test.shape[0]})")


Données prêtes pour l'entraînement (train: 643, test: 161)


**Entraînement et évaluation des modèles**

In [24]:
## Entraînement et évaluation des modèles

models = {
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(random_state=42),
    "SVM": SVC(random_state=42),
    "KNN": KNeighborsClassifier(),
}

model_scores = {}

for name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring="accuracy")
    mean_score = scores.mean()
    model_scores[name] = mean_score
    print(f"{name} - Précision moyenne : {mean_score:.4f}")

# Meilleur modèle
best_model_name = max(model_scores, key=model_scores.get)
best_score = model_scores[best_model_name]

print(f"Meilleur modèle : {best_model_name} avec une précision de {best_score:.4f}")


Decision Tree - Précision moyenne : 0.9564
Random Forest - Précision moyenne : 0.9564
Logistic Regression - Précision moyenne : 0.9735
SVM - Précision moyenne : 0.9720
KNN - Précision moyenne : 0.9688
Meilleur modèle : Logistic Regression avec une précision de 0.9735


**Entraînement final et exportation du modèle**

In [25]:
## Entraînement final et exportation du modèle

# Entraîner sur train
best_model = models[best_model_name]
best_model.fit(X_train, y_train)

# Évaluer sur test
test_score = best_model.score(X_test, y_test)
print(f"Précision sur l'ensemble de test : {test_score:.4f}")

# Sauvegarde du modèle et du scaler
joblib.dump(best_model, "colon_cancer_model.pkl")
joblib.dump(scaler, "scaler.pkl")

print("Modèle et scaler sauvegardés sous 'colon_cancer_model.pkl' et 'scaler.pkl'")


Précision sur l'ensemble de test : 1.0000
Modèle et scaler sauvegardés sous 'colon_cancer_model.pkl' et 'scaler.pkl'


**Télécharger le modele**

In [26]:
from google.colab import files

files.download('colon_cancer_model.pkl')
files.download('scaler.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>